# ETL LegalRES

* Extract: Bajar los JSON de legalRES de S3 
* Transform: Seleccionar las columnas que interesan, aplanar a una tabla y generar columnas nuevas
* Load: Cargar tabla limpia a S3

In [74]:
%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
#!pip install awswrangler
#!pip install --upgrade pandas

In [76]:
import sys
sys.path.append('../../src/data')

from make_dataset import *

import pandas as pd
import boto3
from tqdm.notebook import tqdm

import awswrangler as wr

import json
from pandas.io.json import json_normalize

#data = pd.read_json(data_location)

In [77]:
USE_LOCAL = True

In [78]:
bucket='legalres-storage-study-json'
data_key = 'AC000lvffws1.json'
data_location = 's3://{}/{}'.format(bucket, data_key)

session = boto3.Session(profile_name='legalbot')
boto3.setup_default_session(profile_name='legalbot')
s3 = boto3.client('s3')


Config datos ya extraidos para no duplicar

In [79]:
prefix_processed = 'processed'
data_location_processed = 's3://{}/{}/'.format(bucket, prefix_processed)

## Extract 

Datos ya extraidos para no duplicar

In [80]:
if USE_LOCAL:
    df_processed = pd.read_csv('../../data/processed/processed-studies.csv')
else:
    df_processed = wr.s3.read_parquet(path=data_location_processed,
                                  #columns=['cve'], 
                                      boto3_session=session )

537735

798060

880520

881320

In [81]:
df_processed.shape

(881320, 7)

In [82]:
if not USE_LOCAL:
    df_processed.to_csv('../../data/processed/processed-studies.csv', index=False)

In [83]:
cve_processed = list(df_processed['cve'].values)

In [84]:
assert len(cve_processed) == len(df_processed.index)

In [85]:
'AC0AEHf1alMr' in cve_processed

True

Prueba con un archivo para ver estructura

Constitucion: AC00DaeUV9O5.json

In [86]:
data_stream = s3.get_object(Bucket=bucket,Key='AC00DaeUV9O5.json')['Body'].read().decode('utf-8')
data = json.loads(data_stream)
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "error_fatal": false,
    "disuelto": false,
    "rut": "76305353-9",
    "fecha": "2013-10-16",
    "cve": "AC00DaeUV9O5",
    "tipo_actuacion": "CONSTITUCION",
    "tipo_sociedad": "EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA",
    "estructura": [
        {
            "numero": 1,
            "tipo": "titulo_primero",
            "texto": "CONSTITUCION DE EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA SIGMAQ MAQUINARIA E INGENIERIA DIEGO IGNACIO ALVIAL ARREPOL E.I.R.L. Rut 76.305.353-9"
        },
        {
            "numero": 2,
            "tipo": "principal",
            "articulos": [
                {
                    "numero": 0,
                    "id": "articulo_0",
                    "articulo": "inicio",
                    "subtitulo": "",
                    "texto": "En PROVIDENCIA, Región METROPOLITANA DE SANTIAGO, Chile, a 16 de octubre del 2013, ante el Registro Electrónico de Empresas y Sociedades, comparece: DIEGO IGNACIO ALVIAL ARREPOL, Rut 14.374.

In [87]:
"\"tipo_actuacion\": \"CONSTITUCION\"" in data_stream[:200]

True

Helper functions

## Get all Keys that should be processeed 

In [88]:
fu = file_utilities()

In [ ]:
all_keys = fu.get_keys_from_prefix(bucket)

In [ ]:
all_keys_set = set([key.split('.')[0] for key in all_keys if 'processed' not in key])

In [ ]:
cve_processed_set = set(cve_processed)

In [ ]:
keys_not_processed = all_keys_set.difference(cve_processed_set)

In [ ]:
keys_not_processed = [str(key) + '.json' for key in keys_not_processed]

In [ ]:
len(keys_not_processed)

In [ ]:
proc = [x for x in keys_not_processed if 'processed' in x]

In [ ]:
assert len(proc) == 0

## Intentando Multiprocess

In [100]:
num_proc = os.cpu_count() #number of processors
print(num_proc)

4


In [77]:
progress_bar = tqdm(total=len(keys_not_processed[:1000]))
df_list = get_dflist_multiprocess(keys_not_processed[:1000], progress_bar, num_proc=num_proc) #collect dataframes for each file


In [78]:
[x for x in df_list if x is not None]

[]

## Proceso normal 

El loop principal que recorre todos los archivos

**Tarda 12 horas en procesar toda la carpeta**

In [ ]:

LIMIT = 1 # early stopping for debugging purposes
i = 0 # pages
j = 0 # constituciones
k = 0
output = []
failed_cve = []

for key in tqdm(keys_not_processed):
    i = i + 1

    # Read data
    data_stream = s3.get_object(Bucket=bucket,Key=key)['Body'].read().decode('utf-8')
    try:
        data = json.loads(data_stream)
        dictout = get_data_study(data)
        output.append(dictout)
        j = j + 1
    except:
        k = k + 1
        failed_cve.append(key)

    if i % 5000 == 0:
        try:
            a = failed_cve[0]
        except:
            a = ''
        print(i, '/ Processed:', j, '| Not Processed:', k, '| Sample', a, 
          '| Total Failed:', len(failed_cve))

    #if i >= LIMIT:
    #    break

5000 / Processed: 4994 | Not Processed: 6 | Sample ACsC7n1517WM.json | Total Failed: 6
10000 / Processed: 9991 | Not Processed: 9 | Sample ACsC7n1517WM.json | Total Failed: 9
15000 / Processed: 14979 | Not Processed: 21 | Sample ACsC7n1517WM.json | Total Failed: 21
20000 / Processed: 19976 | Not Processed: 24 | Sample ACsC7n1517WM.json | Total Failed: 24
25000 / Processed: 24969 | Not Processed: 31 | Sample ACsC7n1517WM.json | Total Failed: 31
30000 / Processed: 29964 | Not Processed: 36 | Sample ACsC7n1517WM.json | Total Failed: 36
35000 / Processed: 34959 | Not Processed: 41 | Sample ACsC7n1517WM.json | Total Failed: 41


In [99]:
df = pd.DataFrame(output)
df.head()

,rut,cve,fecha,actuacion,lugar,tipo_sociedad,objeto,firmas
0,76503991-6,ACO0GXJnDTVe,2016-11-07,MODIFICACION,None,None,None,None
1,76562099-6,ACDXQGePMB0v,2017-05-12,MODIFICACION,None,None,None,None
2,77211978-K,ACcS2XsEmQgC,2021-08-20,MODIFICACION,None,None,None,None
3,76375118-K,ACriiIqPeht6,2014-08-11,MODIFICACION,None,None,None,None
4,76342855-9,ACzC1vgrKsPV,2017-12-18,MODIFICACION,None,None,None,None


In [100]:
df.shape

(217, 8)

In [109]:
df.dtypes

Series([], dtype: object)

## Transform 

Campo fecha

In [106]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%Y-%m-%d")

KeyError: 'fecha'

In [ ]:
df['year'] = df['fecha'].dt.year
df['month'] = df['fecha'].dt.month

In [ ]:
df.head()

## Load 

En formato Apache Parquet

In [21]:


wr.s3.to_parquet(
    df,
    path='s3://{}/processed'.format(bucket),
    dataset=True,
    partition_cols=['year', 'month'],
    mode='append'
)

{'paths': ['s3://legalres-storage-study-json/processed/year=2013/month=5/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=7/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=8/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=10/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=11/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2013/month=12/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=1/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=2/275de328138f41dfb2cbd75ced977514.snappy.parquet',
  's3://legalres-storage-study-json/processed/year=2014/month=3/275de328138f